In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

import warnings
warnings.filterwarnings("ignore")

### 1. Load Data

In [2]:
pwd

"/Users/ChunyanHao/Desktop/GitHub/DS_Take_Home/Bella's Code"

In [36]:
homepage = pd.read_csv('/Users/ChunyanHao/Desktop/GitHub/DS_Take_Home/data/Funnel/home_page_table.csv')
payment_cfm = pd.read_csv('/Users/ChunyanHao/Desktop/GitHub/DS_Take_Home/data/Funnel/payment_confirmation_table.csv')
payment_page = pd.read_csv('/Users/ChunyanHao/Desktop/GitHub/DS_Take_Home/data/Funnel/payment_page_table.csv')
search_page = pd.read_csv('/Users/ChunyanHao/Desktop/GitHub/DS_Take_Home/data/Funnel/search_page_table.csv')
users = pd.read_csv('/Users/ChunyanHao/Desktop/GitHub/DS_Take_Home/data/Funnel/user_table.csv')

In [14]:
file_list = [homepage, payment_cfm, payment_page, search_page, users]
for i in file_list:
    print(i.head(3))

   user_id       page
0   313593  home_page
1   468315  home_page
2   264005  home_page
   user_id                       page
0   123100  payment_confirmation_page
1   704999  payment_confirmation_page
2   407188  payment_confirmation_page
   user_id          page
0   253019  payment_page
1   310478  payment_page
2   304081  payment_page
   user_id         page
0    15866  search_page
1   347058  search_page
2   577020  search_page
   user_id        date   device     sex
0   450007  2015-02-28  Desktop  Female
1   756838  2015-01-13  Desktop    Male
2   568983  2015-04-09  Desktop    Male


In [47]:
file_list = [users, homepage, search_page, payment_page, payment_cfm]
for i in file_list:
    print(i.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90400 entries, 0 to 90399
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   user_id       90400 non-null  int64  
 1   date          90400 non-null  object 
 2   device        90400 non-null  object 
 3   sex           90400 non-null  object 
 4   homepage      90400 non-null  int64  
 5   search_page   90400 non-null  float64
 6   payment_page  90400 non-null  float64
 7   payment_cfm   90400 non-null  float64
dtypes: float64(3), int64(2), object(3)
memory usage: 6.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90400 entries, 0 to 90399
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   user_id  90400 non-null  int64
 1   page     90400 non-null  int64
dtypes: int64(2)
memory usage: 1.4 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45200 entries, 0 to 45199
Data columns (total 2 co

### 2. Merge data and calculate basic conversion rate

In [37]:
file_list = [homepage, payment_cfm, payment_page, search_page]
for i in file_list:
    i['page'] = 1

In [38]:
file_list = [homepage, search_page, payment_page, payment_cfm]
for i in file_list:
    users = pd.merge(left = users, right = i, how = 'left', 
                     left_on = 'user_id', right_on = 'user_id')
users.columns = ['user_id', 'date', 'device', 'sex', 
                 'homepage', 'search_page', 'payment_page', 'payment_cfm']
users = users.fillna(0)

/Users/ChunyanHao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'page_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.


In [40]:
users.head(3)

,user_id,date,device,sex,homepage,search_page,payment_page,payment_cfm
0,450007,2015-02-28,Desktop,Female,1,0.0,0.0,0.0
1,756838,2015-01-13,Desktop,Male,1,0.0,0.0,0.0
2,568983,2015-04-09,Desktop,Male,1,1.0,0.0,0.0


In [54]:
# 所有的用户中有多少进入这一页
# calculate the general conversion rate of all user to next page
# for example: 50% of all users reach out the search_page, 
# but only 0.5% of all users reach out the payment_confirmation page

for i in ['homepage', 'search_page', 'payment_page', 'payment_cfm']:
    print('homepage -->', i, np.mean(users[i]))

homepage --> homepage 1.0
homepage --> search_page 0.5
homepage --> payment_page 0.06670353982300885
homepage --> payment_cfm 0.005


In [55]:
# 当前页中有多少进入了最后一页
# calculate the conversion rate in each page to final payment_cfm base on previous page
# for example: 0.5% of all users in homepage reach out the payment_confrimation page, 
# but 1% of all users in search_page reach out the payment_confirmation page
# also, 7.5% of all users in payment_page reach out the payment_confirmation page

for i in ['homepage', 'search_page', 'payment_page', 'payment_cfm']:
    print(i, '--> payment_cfm', users[(users[i] == 1)]['payment_cfm'].mean())

homepage --> payment_cfm 0.005
search_page --> payment_cfm 0.01
payment_page --> payment_cfm 0.07495854063018242
payment_cfm --> payment_cfm 1.0


In [56]:
# 当前页中有多少进入了下一页
# calculate the conversion rate in each page to next page
# for example: 50% of all users in homepage reach out the search page, 
# but only 13% of all users in search_page reach out the payment page
# also, only 7.5% of all users in payment_page reach out the payment_confirmation page

print('homepage --> search_page', users[(users['homepage'] == 1)]['search_page'].mean())
print('search_page --> payment_page', users[(users['search_page'] == 1)]['payment_page'].mean())
print('payment_page --> payment_cfm', users[(users['payment_page'] == 1)]['payment_cfm'].mean())

homepage --> search_page 0.5
search_page --> payment_page 0.1334070796460177
payment_page --> payment_cfm 0.07495854063018242


### 3. Conversion rate by device and sex

In [70]:
temp_1 = users[(users['homepage'] == 1)].groupby('device')['search_page'].mean().reset_index()
temp_2 = users[(users['search_page'] == 1)].groupby('device')['payment_page'].mean().reset_index()
temp_3 = users[(users['payment_page'] == 1)].groupby('device')['payment_cfm'].mean().reset_index()
device_summary = temp_1.merge(temp_2, on = 'device').merge(temp_3, on = 'device')
device_summary

,device,search_page,payment_page,payment_cfm
0,Desktop,0.5,0.1,0.049834
1,Mobile,0.5,0.2,0.100000


##### the performance of mobile users are abvious better than the desktop users in conversion rate. This means that the company needs more modification in their website. Meanwhile, it's strange that the conversion rate from payment page to panyment_confirmation page is very low. That indecates many customers change their mind to confirm the final order click.

In [71]:
temp_1 = users[(users['homepage'] == 1)].groupby('sex')['search_page'].mean().reset_index()
temp_2 = users[(users['search_page'] == 1)].groupby('sex')['payment_page'].mean().reset_index()
temp_3 = users[(users['payment_page'] == 1)].groupby('sex')['payment_cfm'].mean().reset_index()
sex_summary = temp_1.merge(temp_2, on = 'sex').merge(temp_3, on = 'sex')
sex_summary

,sex,search_page,payment_page,payment_cfm
0,Female,0.503073,0.136708,0.077742
1,Male,0.496944,0.130083,0.072014


##### There is no significant difference between female and male users in conversion rate